USGS Io database: https://pubs.usgs.gov/sim/3168/Io_SIM3168_Database.zip

```
conda environment: research3

geopandas : 0.9.0
logging   : 0.5.1.2
geoplot   : 0.5.1
numpy     : 1.23.5
astropy   : 5.2.1
matplotlib: 3.6.2
shapefile : 2.3.1
jdaviz    : 3.2.dev282+g6cb1b51d
pandas    : 1.5.2
shapely   : 1.8.4
cartopy   : 0.21.1
```

In [ ]:
import warnings
import logging
import tempfile
from functools import wraps

%matplotlib inline
import matplotlib
from matplotlib.image import imread
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes

import numpy as np
from glue.core.roi import XRangeROI

import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.io import fits
from astropy.table import QTable
from astropy.time import Time
from astropy.nddata import NDDataArray
from astropy.utils.masked import Masked
from astropy.wcs import WCS

from astroquery.jplhorizons import Horizons
from astroquery.mast import Observations

from regions import PixCoord, CirclePixelRegion
from specutils import Spectrum1D, SpectralRegion

import matplotlib.pyplot as plt
from matplotlib.colors import to_hex

from jdaviz import Cubeviz, Imviz


import pandas as pd
import shapefile as shp

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

from shapely import wkt
import geopandas as gpd

import geoplot

from descartes import PolygonPatch

In [ ]:
# from watermark import watermark

# print(watermark(iversions=True, globals_=globals(), conda=True))

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

ax.imshow(plt.imread('data/io_map.png'), origin='upper')
imviz_lims = dict(
    xlim=[800, 1550],
    ylim=[200, 900],
)
ax.set(**imviz_lims)
ax.axis('off')

ax.set_aspect('equal')

In [ ]:
data_dir = tempfile.gettempdir()

fn = "jw01373-o031_t007_miri_ch1-shortmediumlong_s3d.fits"  # io
local_path = f'{data_dir}/{fn}'
result = Observations.download_file(f"mast:JWST/product/{fn}", local_path=local_path)

header = fits.getheader(local_path, ext=0)

# observing beginning/end times are in the FITS header:
obs_beg = Time(header['DATE-BEG'], format='isot', scale='utc')
# obs_end = Time(header, format='mjd', scale='utc')

# set up a Horizons query
io_jwst = Horizons(
    # Jupiter's moon Io:
    id="501",
    # JWST's coordinates (in flight):
    location="500@-170",
    # return ephemeris at 1 min intervals during obs:
    epochs=dict(
        start=obs_beg.utc.iso,
        stop=(obs_beg + (1*u.s).to(u.day)).utc.iso,
        step='1m'
    )
)

# select "quantities" to query from these columns, as defined
# by these docs: https://ssd.jpl.nasa.gov/horizons/manual.html#obsquan
ephemeris = io_jwst.ephemerides(quantities=','.join(map(str, np.arange(1, 44))), extra_precision=True)
target_radius = float(io_jwst.last_response.text.split(
    'Mean radius (km)'
)[1].strip()[2:].split()[0]) * u.km
target_distance = ephemeris['delta'].quantity[0]

def get_cols_from_ephem(ephem, cols):
    return map(
        lambda x: x.quantity[0], 
        ephem[cols].itercols()
    )

lon_sub_solar, lat_sub_solar = get_cols_from_ephem(ephemeris, ['PDSunLon', 'PDSunLat'])

lon_sub_observer, lat_sub_observer = get_cols_from_ephem(ephemeris, ['PDObsLon', 'PDObsLat'])

# lons defined as west-positive for Io

lon_sub_solar = 360*u.deg - lon_sub_solar
lon_sub_observer = 360*u.deg - lon_sub_observer

# north_pole_ra, north_pole_dec = get_cols_from_ephem(ephemeris, ['NPole_RA', 'NPole_DEC',])

In [ ]:
dirname = 'Io_SIM3168_Database/Io_Geology_SIM3168_shapefiles/'
files = [
    'Io_HotSpots_Locations.shp',
    'Io_VolcanicCenters.shp',
    'Io_Structure.shp'
]

features = []
for fn in files:
    df = gpd.read_file(dirname + fn)
    
    def wkt_loads(x):
        try:
            return wkt.loads(x)
        except Exception:
            return None

    df = df.dropna(subset=['geometry'])
    features.append(df)
    # display(df)

for i in range(3):
    features[i]['coords'] = features[i]['geometry'].apply(lambda x: x.representative_point().coords[:][0])

In [ ]:
def fill_dark_side(ax, lat, lon, globe, **kwargs):
    """
    Plot a fill on the dark side of the planet (without refraction).

    Parameters
    ----------
        ax : matplotlib axes
            The axes to plot on.
        time : datetime
            The time to calculate terminator for. Defaults to datetime.utcnow()
        **kwargs :
            Passed on to Matplotlib's ax.fill()

    """
    pole_lng = lon
    if lat > 0:
        pole_lat = -90 + lat
        central_rot_lng = 180
    else:
        pole_lat = 90 + lat
        central_rot_lng = 0

    rotated_pole = ccrs.RotatedPole(pole_latitude=pole_lat,
                                    pole_longitude=pole_lng,
                                   central_rotated_longitude=central_rot_lng, globe=globe)

    x = np.empty(360)
    y = np.empty(360)
    x[:180] = -90
    y[:180] = np.arange(-90, 90.)
    x[180:] = 90
    y[180:] = np.arange(90, -90., -1)

    ax.fill(x, y, transform=rotated_pole, **kwargs)

def rebin(a, shape):
    sh = shape[0], a.shape[0]//shape[0], shape[1], a.shape[1]//shape[1]
    return a.reshape(sh).mean(-1).mean(1).astype(int)

In [ ]:
mercator_path = (
    "Io_SIM3168_Database/raster_baseMaps/"
    "Mercator_clon180/Io_SSI_VGR_color_merge_merc180.tif"
)
basemap_mercator = imread(mercator_path)

crop_shape = (4400, 11400)
new_shape = np.array(crop_shape) // 2
basemap_binned = np.transpose(
    [rebin(bm[:crop_shape[0], :crop_shape[1]], new_shape) 
     for bm in basemap_mercator.transpose(2, 0, 1)], (1, 2, 0)
).astype(basemap_mercator.dtype)

In [ ]:
visible_lon_range = Angle([lon_sub_observer - 90*u.deg, lon_sub_observer + 90*u.deg])

needs_wrap_handling = visible_lon_range.max() > 360*u.deg or visible_lon_range.min() < 0*u.deg

if needs_wrap_handling:
    wrap_at = 180*u.deg
    visible_lon_range = visible_lon_range.wrap_at(wrap_at)
    
visible_lon_range

In [ ]:
cmap = plt.cm.rainbow

binedges = np.linspace(4.9004, 7.65, n_subsets + 1)
wl_centroids = 0.5 * (binedges[:-1] + binedges[1:]) * u.um

# number of spectral subsets to assign to colors:
n_subsets = 5

# colormap to adopt:
cmap = plt.cm.rainbow

# get hex colors for each subset
hex_colors = [
    [wl_cen, to_hex(c)] for wl_cen, c in 
    zip(
        wl_centroids, cmap(np.linspace(0, 1, n_subsets))
    )
]

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
params = {'font.family': 'serif'} # , 'text.usetex': True
plt.rcParams.update(params)

fig = plt.figure(figsize=(10, 4))
ax_jdaviz = fig.add_subplot(121)
source_img = plt.imread('data/io_map.png') #np.flipud()
im = ax_jdaviz.imshow(source_img, origin='upper')

transform = Affine2D().translate(
    -source_img.shape[1]/2, -source_img.shape[0]/2
).rotate_deg(
    -1 * u.Quantity(ephemeris['sat_PANG'])[0].to_value(u.deg)
)
trans_data = transform + ax_jdaviz.transData
im.set_transform(trans_data)
ax_jdaviz.set(
    xlim=np.array([-500, 500]) + 100,
    ylim=[-500, 500],
    xticks=[],
    yticks=[]
)

ax_jdaviz.set_facecolor("k")
ax_jdaviz.annotate(
    "Io (JWST/MIRI)\n" + obs_beg.iso[:-7],
    (0.05, 0.05), color='w', 
    xycoords='axes fraction'
)


label_colors = [(111, 121, 181), (215, 134, 101)]
label_positions = [[0.1, 0.8], [0.7, 0.2]]

for pos, wl_cen, color in zip(label_positions, [wl_centroids[0], wl_centroids[-1]], label_colors):
    ax_jdaviz.annotate(
        f"{wl_cen.to_value(u.um):.2f} µm",
        pos, color=to_hex(np.array(color)/256), fontsize=14,
        xycoords='axes fraction'
    )

extent = [visible_lon_range[0].deg, visible_lon_range[1].deg, -90, 90]

globe = ccrs.Globe()
projection = ccrs.NearsidePerspective(
    central_longitude=lon_sub_observer.to_value(u.deg), 
    central_latitude=lat_sub_observer.to_value(u.deg), 
    globe=globe, 
    satellite_height=target_distance.to_value(u.m)
)

ax = fig.add_subplot(122, projection=projection)
data_crs = ccrs.PlateCarree()

annotation_kwargs = dict(
    color='k',
    ha='center',
    va='center', 
    transform=data_crs
)
ax.set_global()


skip_counter = 0
skip_every = 3
for i in [0, 1]:
    visible_geometries = []
    for geom, row in zip(shpreader.Reader(dirname+files[i]).geometries(), features[i].iterrows()):
        long = Angle(row[1]['coords'][0]*u.deg).wrap_at(wrap_at)
                
        if (
            long > visible_lon_range[0] + 20 * u.deg and 
            long < visible_lon_range[1] - 20 * u.deg and 
            not (row[1][0].startswith('Unnamed') or row[1][0].startswith('Unamed'))
        ):
            visible_geometries.append(geom)
    
            fontsize = int(
                6 + 3 * np.log10(row[1]['SHAPE_Area']) 
                if i == 1 else 6
            )
            alpha = 1 if i == 1 else 0.75
            
            skip_counter += 1
            if skip_counter % skip_every == 0 or i:
                ax.annotate(
                    '\n'.join(row[1][0].split()), row[1]['coords'], 
                    fontsize=fontsize, alpha=alpha, **annotation_kwargs
                )
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        ax.add_geometries(visible_geometries, data_crs, fc='none', ec='k', lw=1, alpha=0.4)

fill_dark_side(
    ax, 
    lat_sub_solar.to_value(u.deg), 
    lon_sub_solar.to_value(u.deg),
    globe=globe,
    alpha=0.3, color='k'
)

ax.imshow(basemap_binned, origin='upper', alpha=0.7,
          transform=ccrs.PlateCarree(central_longitude=180, globe=globe),
          extent=[-180, 180, -90, 90], zorder=-5, rasterized=True)
ax.gridlines(crs=ccrs.PlateCarree(), ls=':', color='k', lw=1)

# plt.scatter(lon_sub_observer, lat_sub_observer, marker='o', transform=ccrs.Geodetic())
# plt.scatter(lon_sub_solar, lat_sub_solar, marker='o', transform=ccrs.Geodetic())

lon_sub_jupiter, lat_sub_jupiter = 0, 0
ax.scatter(lon_sub_jupiter, lat_sub_jupiter, marker='x', transform=ccrs.Geodetic(), color='w', s=50)
ax.margins(0.3, tight=None)
fig.subplots_adjust(wspace=-0.1)
fig.savefig('plots/side-by-side.pdf', bbox_inches='tight', dpi=300)
plt.show()
plt.rcParams.update(plt.rcParamsDefault)